In [1]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor

torch.manual_seed(1)

In [2]:
class PosDatasetReader(DatasetReader):
    """
    DatasetReader for PoS tagging data, one sentence per line, like

        The###DET dog###NN ate###V the###DET apple###NN
    """
    def __init__(self, token_indexers: Dict[str, TokenIndexer] = None) -> None:
        super().__init__(lazy=False)
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        
    def text_to_instance(self, tokens: List[Token], tags: List[str] = None) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"sentence": sentence_field}

        if tags:
            label_field = SequenceLabelField(labels=tags, sequence_field=sentence_field)
            fields["labels"] = label_field

        return Instance(fields)
    
    def _read(self, file_path: str) -> Iterator[Instance]:
        with open(file_path) as f:
            for line in f:
                pairs = line.strip().split()
                sentence, tags = zip(*(pair.split("###") for pair in pairs))
                yield self.text_to_instance([Token(word) for word in sentence], tags)

In [3]:
class LstmTagger(Model):
    def __init__(self,
                 word_embeddings: TextFieldEmbedder,
                 encoder: Seq2SeqEncoder,
                 vocab: Vocabulary) -> None:
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                          out_features=vocab.get_vocab_size('labels'))
        self.accuracy = CategoricalAccuracy()
    def forward(self,
                sentence: Dict[str, torch.Tensor],
                labels: torch.Tensor = None) -> Dict[str, torch.Tensor]:
        mask = get_text_field_mask(sentence)
        embeddings = self.word_embeddings(sentence)
        encoder_out = self.encoder(embeddings, mask)
        tag_logits = self.hidden2tag(encoder_out)
        output = {"tag_logits": tag_logits}
        if labels is not None:
            self.accuracy(tag_logits, labels, mask)
            output["loss"] = sequence_cross_entropy_with_logits(tag_logits, labels, mask)

        return output
    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {"accuracy": self.accuracy.get_metric(reset)}

In [4]:
reader = PosDatasetReader()
train_dataset = reader.read(cached_path(
    'https://raw.githubusercontent.com/allenai/allennlp'
    '/master/tutorials/tagger/training.txt'))
validation_dataset = reader.read(cached_path(
    'https://raw.githubusercontent.com/allenai/allennlp'
    '/master/tutorials/tagger/validation.txt'))
vocab = Vocabulary.from_instances(train_dataset + validation_dataset)


93B [00:00, 81553.48B/s]             
2it [00:00, 8839.42it/s]
93B [00:00, 83724.03B/s]             
2it [00:00, 7281.78it/s]
100%|██████████| 4/4 [00:00<00:00, 30066.70it/s]


In [5]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)

if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1

optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("sentence", "num_tokens")])
iterator.index_with(vocab)

trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=1000,
                  cuda_device=cuda_device)

trainer.train()
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)
tag_logits = predictor.predict("The dog ate the apple")['tag_logits']
tag_ids = np.argmax(tag_logits, axis=-1)

accuracy: 0.3333, loss: 1.1685 ||: 100%|██████████| 1/1 [00:00<00:00,  1.60it/s]
accuracy: 0.3333, loss: 1.1592 ||: 100%|██████████| 1/1 [00:00<00:00, 36.34it/s]
accuracy: 0.3333, loss: 1.1604 ||: 100%|██████████| 1/1 [00:00<00:00, 62.36it/s]
accuracy: 0.3333, loss: 1.1516 ||: 100%|██████████| 1/1 [00:00<00:00, 289.22it/s]
accuracy: 0.3333, loss: 1.1529 ||: 100%|██████████| 1/1 [00:00<00:00, 45.65it/s]
accuracy: 0.3333, loss: 1.1445 ||: 100%|██████████| 1/1 [00:00<00:00, 299.76it/s]
accuracy: 0.3333, loss: 1.1458 ||: 100%|██████████| 1/1 [00:00<00:00, 52.91it/s]
accuracy: 0.3333, loss: 1.1379 ||: 100%|██████████| 1/1 [00:00<00:00, 233.57it/s]
accuracy: 0.3333, loss: 1.1391 ||: 100%|██████████| 1/1 [00:00<00:00, 48.10it/s]
accuracy: 0.3333, loss: 1.1316 ||: 100%|██████████| 1/1 [00:00<00:00, 74.05it/s]
accuracy: 0.3333, loss: 1.1329 ||: 100%|██████████| 1/1 [00:00<00:00, 48.44it/s]
accuracy: 0.3333, loss: 1.1259 ||: 100%|██████████| 1/1 [00:00<00:00, 314.96it/s]
accuracy: 0.3333, loss: 

accuracy: 0.4444, loss: 1.0534 ||: 100%|██████████| 1/1 [00:00<00:00, 52.95it/s]
accuracy: 0.4444, loss: 1.0517 ||: 100%|██████████| 1/1 [00:00<00:00, 329.22it/s]
accuracy: 0.4444, loss: 1.0531 ||: 100%|██████████| 1/1 [00:00<00:00, 67.19it/s]
accuracy: 0.4444, loss: 1.0513 ||: 100%|██████████| 1/1 [00:00<00:00, 234.32it/s]
accuracy: 0.4444, loss: 1.0528 ||: 100%|██████████| 1/1 [00:00<00:00, 56.69it/s]
accuracy: 0.4444, loss: 1.0510 ||: 100%|██████████| 1/1 [00:00<00:00, 349.64it/s]
accuracy: 0.4444, loss: 1.0524 ||: 100%|██████████| 1/1 [00:00<00:00, 61.46it/s]
accuracy: 0.4444, loss: 1.0507 ||: 100%|██████████| 1/1 [00:00<00:00, 126.48it/s]
accuracy: 0.4444, loss: 1.0521 ||: 100%|██████████| 1/1 [00:00<00:00, 66.02it/s]
accuracy: 0.4444, loss: 1.0504 ||: 100%|██████████| 1/1 [00:00<00:00, 315.12it/s]
accuracy: 0.4444, loss: 1.0518 ||: 100%|██████████| 1/1 [00:00<00:00, 48.39it/s]
accuracy: 0.4444, loss: 1.0501 ||: 100%|██████████| 1/1 [00:00<00:00, 327.04it/s]
accuracy: 0.4444, loss

accuracy: 0.4444, loss: 1.0370 ||: 100%|██████████| 1/1 [00:00<00:00, 74.03it/s]
accuracy: 0.4444, loss: 1.0353 ||: 100%|██████████| 1/1 [00:00<00:00, 120.06it/s]
accuracy: 0.4444, loss: 1.0366 ||: 100%|██████████| 1/1 [00:00<00:00, 74.65it/s]
accuracy: 0.4444, loss: 1.0349 ||: 100%|██████████| 1/1 [00:00<00:00, 114.04it/s]
accuracy: 0.4444, loss: 1.0362 ||: 100%|██████████| 1/1 [00:00<00:00, 53.86it/s]
accuracy: 0.4444, loss: 1.0345 ||: 100%|██████████| 1/1 [00:00<00:00, 423.50it/s]
accuracy: 0.4444, loss: 1.0357 ||: 100%|██████████| 1/1 [00:00<00:00, 54.63it/s]
accuracy: 0.4444, loss: 1.0341 ||: 100%|██████████| 1/1 [00:00<00:00, 450.27it/s]
accuracy: 0.4444, loss: 1.0353 ||: 100%|██████████| 1/1 [00:00<00:00, 55.44it/s]
accuracy: 0.4444, loss: 1.0336 ||: 100%|██████████| 1/1 [00:00<00:00, 445.59it/s]
accuracy: 0.4444, loss: 1.0349 ||: 100%|██████████| 1/1 [00:00<00:00, 60.64it/s]
accuracy: 0.4444, loss: 1.0332 ||: 100%|██████████| 1/1 [00:00<00:00, 315.65it/s]
accuracy: 0.4444, loss

accuracy: 0.4444, loss: 1.0061 ||: 100%|██████████| 1/1 [00:00<00:00, 87.54it/s]
accuracy: 0.4444, loss: 1.0042 ||: 100%|██████████| 1/1 [00:00<00:00, 337.19it/s]
accuracy: 0.4444, loss: 1.0051 ||: 100%|██████████| 1/1 [00:00<00:00, 142.12it/s]
accuracy: 0.4444, loss: 1.0033 ||: 100%|██████████| 1/1 [00:00<00:00, 329.97it/s]
accuracy: 0.4444, loss: 1.0042 ||: 100%|██████████| 1/1 [00:00<00:00, 87.62it/s]
accuracy: 0.4444, loss: 1.0024 ||: 100%|██████████| 1/1 [00:00<00:00, 331.43it/s]
accuracy: 0.4444, loss: 1.0033 ||: 100%|██████████| 1/1 [00:00<00:00, 144.66it/s]
accuracy: 0.4444, loss: 1.0014 ||: 100%|██████████| 1/1 [00:00<00:00, 332.20it/s]
accuracy: 0.4444, loss: 1.0023 ||: 100%|██████████| 1/1 [00:00<00:00, 128.11it/s]
accuracy: 0.4444, loss: 1.0005 ||: 100%|██████████| 1/1 [00:00<00:00, 333.78it/s]
accuracy: 0.4444, loss: 1.0014 ||: 100%|██████████| 1/1 [00:00<00:00, 118.44it/s]
accuracy: 0.4444, loss: 0.9995 ||: 100%|██████████| 1/1 [00:00<00:00, 331.72it/s]
accuracy: 0.4444, 

accuracy: 0.4444, loss: 0.9355 ||: 100%|██████████| 1/1 [00:00<00:00, 155.54it/s]
accuracy: 0.4444, loss: 0.9329 ||: 100%|██████████| 1/1 [00:00<00:00, 312.08it/s]
accuracy: 0.4444, loss: 0.9334 ||: 100%|██████████| 1/1 [00:00<00:00, 174.80it/s]
accuracy: 0.4444, loss: 0.9307 ||: 100%|██████████| 1/1 [00:00<00:00, 443.65it/s]
accuracy: 0.4444, loss: 0.9313 ||: 100%|██████████| 1/1 [00:00<00:00, 171.71it/s]
accuracy: 0.4444, loss: 0.9286 ||: 100%|██████████| 1/1 [00:00<00:00, 442.20it/s]
accuracy: 0.4444, loss: 0.9292 ||: 100%|██████████| 1/1 [00:00<00:00, 167.77it/s]
accuracy: 0.4444, loss: 0.9264 ||: 100%|██████████| 1/1 [00:00<00:00, 309.02it/s]
accuracy: 0.4444, loss: 0.9270 ||: 100%|██████████| 1/1 [00:00<00:00, 128.98it/s]
accuracy: 0.4444, loss: 0.9242 ||: 100%|██████████| 1/1 [00:00<00:00, 310.05it/s]
accuracy: 0.5556, loss: 0.9248 ||: 100%|██████████| 1/1 [00:00<00:00, 104.71it/s]
accuracy: 0.4444, loss: 0.9220 ||: 100%|██████████| 1/1 [00:00<00:00, 236.63it/s]
accuracy: 0.5556

...

accuracy: 1.0000, loss: 0.0298 ||: 100%|██████████| 1/1 [00:00<00:00, 64.14it/s]
accuracy: 1.0000, loss: 0.0296 ||: 100%|██████████| 1/1 [00:00<00:00, 266.98it/s]
accuracy: 1.0000, loss: 0.0297 ||: 100%|██████████| 1/1 [00:00<00:00, 67.75it/s]
accuracy: 1.0000, loss: 0.0295 ||: 100%|██████████| 1/1 [00:00<00:00, 115.81it/s]
accuracy: 1.0000, loss: 0.0296 ||: 100%|██████████| 1/1 [00:00<00:00, 64.91it/s]
accuracy: 1.0000, loss: 0.0294 ||: 100%|██████████| 1/1 [00:00<00:00, 113.06it/s]
accuracy: 1.0000, loss: 0.0295 ||: 100%|██████████| 1/1 [00:00<00:00, 53.11it/s]
accuracy: 1.0000, loss: 0.0293 ||: 100%|██████████| 1/1 [00:00<00:00, 112.83it/s]
accuracy: 1.0000, loss: 0.0294 ||: 100%|██████████| 1/1 [00:00<00:00, 45.41it/s]
accuracy: 1.0000, loss: 0.0293 ||: 100%|██████████| 1/1 [00:00<00:00, 108.61it/s]
accuracy: 1.0000, loss: 0.0293 ||: 100%|██████████| 1/1 [00:00<00:00, 71.24it/s]
accuracy: 1.0000, loss: 0.0292 ||: 100%|██████████| 1/1 [00:00<00:00, 116.27it/s]
accuracy: 1.0000, loss

accuracy: 1.0000, loss: 0.0258 ||: 100%|██████████| 1/1 [00:00<00:00, 174.25it/s]
accuracy: 1.0000, loss: 0.0257 ||: 100%|██████████| 1/1 [00:00<00:00, 439.29it/s]
accuracy: 1.0000, loss: 0.0257 ||: 100%|██████████| 1/1 [00:00<00:00, 172.89it/s]
accuracy: 1.0000, loss: 0.0256 ||: 100%|██████████| 1/1 [00:00<00:00, 380.64it/s]
accuracy: 1.0000, loss: 0.0256 ||: 100%|██████████| 1/1 [00:00<00:00, 48.84it/s]
accuracy: 1.0000, loss: 0.0255 ||: 100%|██████████| 1/1 [00:00<00:00, 151.50it/s]
accuracy: 1.0000, loss: 0.0256 ||: 100%|██████████| 1/1 [00:00<00:00, 175.02it/s]
accuracy: 1.0000, loss: 0.0255 ||: 100%|██████████| 1/1 [00:00<00:00, 416.76it/s]
accuracy: 1.0000, loss: 0.0255 ||: 100%|██████████| 1/1 [00:00<00:00, 176.63it/s]
accuracy: 1.0000, loss: 0.0254 ||: 100%|██████████| 1/1 [00:00<00:00, 437.45it/s]
accuracy: 1.0000, loss: 0.0254 ||: 100%|██████████| 1/1 [00:00<00:00, 75.32it/s]
accuracy: 1.0000, loss: 0.0253 ||: 100%|██████████| 1/1 [00:00<00:00, 443.28it/s]
accuracy: 1.0000, 

accuracy: 1.0000, loss: 0.0226 ||: 100%|██████████| 1/1 [00:00<00:00, 171.25it/s]
accuracy: 1.0000, loss: 0.0226 ||: 100%|██████████| 1/1 [00:00<00:00, 220.34it/s]
accuracy: 1.0000, loss: 0.0226 ||: 100%|██████████| 1/1 [00:00<00:00, 162.00it/s]
accuracy: 1.0000, loss: 0.0225 ||: 100%|██████████| 1/1 [00:00<00:00, 371.90it/s]
accuracy: 1.0000, loss: 0.0225 ||: 100%|██████████| 1/1 [00:00<00:00, 162.10it/s]
accuracy: 1.0000, loss: 0.0224 ||: 100%|██████████| 1/1 [00:00<00:00, 357.72it/s]
accuracy: 1.0000, loss: 0.0225 ||: 100%|██████████| 1/1 [00:00<00:00, 167.79it/s]
accuracy: 1.0000, loss: 0.0224 ||: 100%|██████████| 1/1 [00:00<00:00, 360.18it/s]
accuracy: 1.0000, loss: 0.0224 ||: 100%|██████████| 1/1 [00:00<00:00, 171.53it/s]
accuracy: 1.0000, loss: 0.0223 ||: 100%|██████████| 1/1 [00:00<00:00, 351.19it/s]
accuracy: 1.0000, loss: 0.0224 ||: 100%|██████████| 1/1 [00:00<00:00, 162.09it/s]
accuracy: 1.0000, loss: 0.0223 ||: 100%|██████████| 1/1 [00:00<00:00, 360.34it/s]
accuracy: 1.0000

accuracy: 1.0000, loss: 0.0201 ||: 100%|██████████| 1/1 [00:00<00:00, 147.12it/s]
accuracy: 1.0000, loss: 0.0201 ||: 100%|██████████| 1/1 [00:00<00:00, 284.51it/s]
accuracy: 1.0000, loss: 0.0201 ||: 100%|██████████| 1/1 [00:00<00:00, 121.91it/s]
accuracy: 1.0000, loss: 0.0200 ||: 100%|██████████| 1/1 [00:00<00:00, 279.19it/s]
accuracy: 1.0000, loss: 0.0200 ||: 100%|██████████| 1/1 [00:00<00:00, 85.21it/s]
accuracy: 1.0000, loss: 0.0200 ||: 100%|██████████| 1/1 [00:00<00:00, 284.98it/s]
accuracy: 1.0000, loss: 0.0200 ||: 100%|██████████| 1/1 [00:00<00:00, 148.77it/s]
accuracy: 1.0000, loss: 0.0199 ||: 100%|██████████| 1/1 [00:00<00:00, 269.45it/s]
accuracy: 1.0000, loss: 0.0199 ||: 100%|██████████| 1/1 [00:00<00:00, 84.83it/s]
accuracy: 1.0000, loss: 0.0199 ||: 100%|██████████| 1/1 [00:00<00:00, 282.98it/s]
accuracy: 1.0000, loss: 0.0199 ||: 100%|██████████| 1/1 [00:00<00:00, 96.05it/s]
accuracy: 1.0000, loss: 0.0198 ||: 100%|██████████| 1/1 [00:00<00:00, 196.84it/s]
accuracy: 1.0000, l

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/texuanw/softwares/anaconda3/lib/python3.6/site-packages/en_core_web_sm -->
/home/texuanw/softwares/anaconda3/lib/python3.6/site-packages/spacy/data/en_core_web_sm
You can now load the model via spacy.load('en_core_web_sm')
['DET', 'NN', 'V', 'DET', 'NN']


In [ ]:
print([model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])

In [6]:
# Here's how to save the model.
with open("/tmp/model.th", 'wb') as f:
    torch.save(model.state_dict(), f)
vocab.save_to_files("/tmp/vocabulary")

# And here's how to reload the model.
vocab2 = Vocabulary.from_files("/tmp/vocabulary")
model2 = LstmTagger(word_embeddings, lstm, vocab2)
with open("/tmp/model.th", 'rb') as f:
    model2.load_state_dict(torch.load(f))
if cuda_device > -1:
    model2.cuda(cuda_device)

predictor2 = SentenceTaggerPredictor(model2, dataset_reader=reader)
tag_logits2 = predictor2.predict("The dog ate the apple")['tag_logits']
np.testing.assert_array_almost_equal(tag_logits2, tag_logits)